In [71]:
import mlflow
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [72]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
project_id = os.environ['DOMINO_PROJECT_ID']
#output_dir = "/mnt/artifacts/llama2-results"
output_dir = f"/artifacts/mlflow/{project_id}/llama2-results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 25

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [74]:
# Load dataset
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=f"/artifacts/mlflow/{project_id}/llama2-model-cache/",
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          cache_dir=f"/artifacts/mlflow/{project_id}/llama2-model-cache/",
                                          trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to=["mlflow"],
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


In [70]:
experiment_name = 'llama2-7b-4bit-lora-ft-v2'
exp = mlflow.set_experiment(experiment_name)
print("Fine-tuning model:")
with mlflow.start_run() as run:
    # Start training - Skipping this step
    #trainer.train()
    # Save trained model
    print(f"Saving Model to {output_dir}/{new_model}")
    trainer.model.save_pretrained(f"{output_dir}/{new_model}")

Fine-tuning model:
Saving Model to /artifacts/mlflow/65cd15a3b25a7b376a398426/llama2-results/llama-2-7b-miniguanaco


In [10]:
f"{output_dir}/{new_model}"

'/artifacts/mlflow/65cd15a3b25a7b376a398426/llama2-results/llama-2-7b-miniguanaco'

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True'
base_model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    cache_dir=f"/artifacts/mlflow/{project_id}/llama2-model-cache/",
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, f"{output_dir}/{new_model}")
model = model.merge_and_unload()

output_merged_dir = f"/artifacts/mlflow/{project_id}/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.save_pretrained(output_merged_dir)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

from transformers import GenerationConfig 

prompt = "What should i do in Paris?"

text = f"<s>[INST] {prompt} [/INST]"

device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)

generation_config = GenerationConfig(
            pad_token_id=tokenizer.pad_token_id,
            max_new_tokens = 200
        )

outputs = model.generate(**inputs, generation_config=generation_config)
llm_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
cleaned_output = llm_output.replace(f"[INST] {prompt} [/INST]", '')
print(cleaned_output)

## Model Class

The `MyModel` is a Pyfunc model that will be registered to MLFLOW

In [78]:
import mlflow

class MyModel(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        model_name = "NousResearch/Llama-2-7b-chat-hf"
        # Activate 4-bit precision base model loading
        use_4bit = True

        # Compute dtype for 4-bit base models
        bnb_4bit_compute_dtype = "float16"

        # Quantization type (fp4 or nf4)
        bnb_4bit_quant_type = "nf4"

        # Activate nested quantization for 4-bit base models (double quantization)
        use_nested_quant = False    
        
        
        # Load the entire model on the GPU 0
        device_map = "auto"
        #cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
        #os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir
        import mlflow
        import os
        import torch
        from datasets import load_dataset
        from transformers import (
            AutoModelForCausalLM,
            AutoTokenizer,
            BitsAndBytesConfig,
            HfArgumentParser,
            TrainingArguments,
            pipeline,
            logging,
        )
        from peft import LoraConfig, PeftModel
        from trl import SFTTrainer
        self.project_id = os.listdir('/artifacts/mlflow')[0]
        # Load the ctranslate model
        model_device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # model_path = '/mnt/artifacts/llama2/final_merged_checkpoint/'        
        #model_path = f'/artifacts/mlflow/{self.project_id}/final_merged_checkpoint/'
        model_path = f"/artifacts/mlflow/{self.project_id}/llama2-model-cache/models--NousResearch--Llama-2-7b-chat-hf/"
        model_device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Load tokenizer and model with QLoRA configuration
        compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

        
        bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

        # load the ctranslate model
        # Load base model
        model = AutoModelForCausalLM.from_pretrained(
        model_path,
        cache_dir=model_path,
        quantization_config=bnb_config,
        device_map=device_map
    )
#         self.generator = ctranslate2.Generator(model_path, device=model_device)
        #self.tokenizer = transformers.AutoTokenizer.from_pretrained(f'/artifacts/mlflow/{self.project_id}/llama-2-7b-miniguanaco')
        #self.tokenizer = transformers.AutoTokenizer.from_pretrained(f"/artifacts/mlflow/{self.project_id}/llama2-model-cache/models--NousResearch--Llama-2-7b-chat-hf/")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.tokenizer.pad_token = tokenizer.eos_token
        self.tokenizer.padding_side = "right"


    prompt_template = f"<s>[INST] {{dialogue}} [/INST]"
    def predict(self, context, model_input, params=None):
        """
        This method generates prediction for the given input.
        """
        prompt = model_input["prompt"][0]

        if prompt is None:
            return 'Please provide a prompt.'
            
        # Construct the prompt for the model
        user_input = prompt_template.format(dialogue=prompt)
    
        tokens = self.tokenizer.convert_ids_to_tokens(self.tokenizer.encode(user_input))
        input_length = len(tokens)
        # new_tokens = round(pct_new_tokens*input_length)
        new_tokens = 750
        tokens_per_sec = 0
        start_time = time.time()
        
        text = f"<s>[INST] {prompt} [/INST]"

        device = "cuda:0"

        inputs = tokenizer(text, return_tensors="pt").to(device)

        generation_config = transformers.GenerationConfig(
                    pad_token_id=tokenizer.pad_token_id,
                    max_new_tokens = 200
                )

        outputs = model.generate(**inputs, generation_config=generation_config)
        llm_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        result = llm_output.replace(f"[INST] {prompt} [/INST]", '')
        return {'text_from_llm': result}

In [63]:
import pandas as pd
import numpy as np
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types import DataType, Schema, ColSpec, ParamSchema, ParamSpec

# Define input and output schema
input_schema = Schema(
    [
        ColSpec(DataType.string, "prompt"),
    ]
)
output_schema = Schema([ColSpec(DataType.string, "text_from_llm")])

parameters = ParamSchema(
    [       
    ]
)

signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=parameters)


# Define input example
input_example = pd.DataFrame({"prompt": ["What is machine learning?"]})

## Create a registered model if it does not exist

In [53]:
import mlflow
client = mlflow.MlflowClient()
model_name="lama2-sft"
registered_model = None
try:
    registered_model = client.create_registered_model(model_name)
except:
     registered_model = client.get_registered_model(model_name)


## Create a Model Version

In [64]:
import mlflow
import torch
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
import nvidia
# Get the current base version of torch that is installed, without specific version modifiers
torch_version = torch.__version__.split("+")[0]
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import peft
import trl

# Start an MLflow run context and log the MPT-7B model wrapper along with the param-included signature to
# allow for overriding parameters at inference time
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        model_name,
        python_model=MyModel(),
        # NOTE: the artifacts dictionary mapping is critical! This dict is used by the load_context() method in our MyModel() class.
        artifacts={"snapshot": '/mnt/'},
        pip_requirements=[
            f"torch=={torch_version}",
            f"transformers=={transformers.__version__}",                        
            f"peft=={peft.__version__}",
            f"trl=={trl.__version__}",            
            "einops",
            "sentencepiece",
        ],
        input_example=input_example,
        signature=signature,
    )
    runs_uri = model_info.model_uri
    print(runs_uri)
    # Create a new model version of the RandomForestRegression model from this run
    
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))

2024/02/14 20:39:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lama2-sft, version 5


runs:/104e1028e51f4955b12fc957afae40e6/lama2-sft
Name: lama2-sft
Version: 5
Description: 
Status: READY
Stage: None


# Testing Model

In [79]:
! rm -rf /tmp/mlflow*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [80]:
# Save the model
mlflow.pyfunc.save_model(path="/tmp/mlflow/", python_model=MyModel())
# Load the model for inference
model = mlflow.pyfunc.load_model("/tmp/mlflow/")
result = model.predict({"prompt" : "Where is Cancun?"})


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024/02/14 20:46:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/mlflow/, flavor: python_function), fall back to return ['cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


OSError: /artifacts/mlflow/65cd15a3b25a7b376a398426/llama2-model-cache/models--NousResearch--Llama-2-7b-chat-hf/ does not appear to have a file named config.json. Checkout 'https://huggingface.co//artifacts/mlflow/65cd15a3b25a7b376a398426/llama2-model-cache/models--NousResearch--Llama-2-7b-chat-hf//None' for available files.